In [1]:
!nvidia-smi

Tue Jun  6 14:34:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install -q langchain transformers sentence_transformers llama-index


In [3]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTVectorStoreIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline

In [4]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())


In [5]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)


In [6]:
text1 = """ McDonald's: A Global Icon of Fast Food

McDonald's, the world's largest fast-food chain, has become an iconic symbol of the global food industry. With its golden arches, cheerful red and yellow logo, and distinctive menu items, McDonald's has permeated almost every corner of the world, serving billions of customers since its inception.

The story of McDonald's began in 1940 when Richard and Maurice McDonald opened a small burger stand in San Bernardino, California. Initially, their restaurant focused on offering a limited menu with a strong emphasis on speed and efficiency. However, it was not until 1955 when Ray Kroc, a visionary entrepreneur, joined forces with the brothers and transformed McDonald's into a franchise business.

Kroc's genius lay in recognizing the potential for standardization and mass production in the fast-food industry. He implemented a system that allowed every McDonald's location to serve the same menu items, maintaining consistency in taste and quality. This system, coupled with an aggressive marketing strategy, propelled McDonald's to unprecedented heights of success.

The McDonald's menu, known for its iconic items such as the Big Mac, Quarter Pounder, and Chicken McNuggets, has evolved over the years to cater to changing consumer preferences. In recent times, the company has also made efforts to introduce healthier options, such as salads and grilled chicken sandwiches, to accommodate a more health-conscious customer base.

One of the key factors contributing to McDonald's global dominance is its commitment to localization. While the core menu remains consistent worldwide, McDonald's adapts its offerings to suit local tastes and cultural preferences. For example, in India, where a significant portion of the population follows vegetarian diets, McDonald's introduced a range of vegetarian options like the McAloo Tikki and the Maharaja Mac.

Beyond its culinary offerings, McDonald's has also made a significant impact on popular culture. The brand's iconic advertising campaigns, featuring memorable characters like Ronald McDonald and catchy slogans like "I'm lovin' it," have permeated television screens and billboards worldwide. McDonald's has also established itself as a community gathering place, hosting birthday parties, sponsoring sports events, and supporting charitable initiatives.

However, McDonald's has not been without its share of controversies. The company has faced criticism for its contribution to the rise in obesity rates, its environmental footprint, and its labor practices. In response, McDonald's has taken steps to address these concerns. They have introduced healthier menu options, made commitments to sustainable sourcing, and increased employee wages in some regions.

Looking ahead, McDonald's continues to innovate and adapt to changing consumer demands. The company has embraced technology by introducing self-ordering kiosks, mobile ordering, and delivery services. Additionally, McDonald's has made significant strides in reducing its environmental impact by setting goals to minimize packaging waste and increase recycling.

In conclusion, McDonald's has emerged as a global icon of fast food, captivating billions of customers worldwide. Its commitment to consistency, localization, and customer satisfaction has made it a dominant force in the industry. While it faces challenges and controversies, McDonald's remains an influential player shaping the fast-food landscape and continuing to evolve to meet the needs of its diverse consumer base."""

In [47]:
#pip install PyPDF2

In [46]:
# # importing required modules
# from PyPDF2 import PdfReader
  
# # creating a pdf reader object
# reader = PdfReader('sbc-sample.pdf')
  
# # Initialize an empty string to store the combined text
# combined_text = ""

# # Iterate over all pages
# for page in reader.pages:
#     # Extracting text from each page
#     text = page.extract_text()
#     # Append the text to the combined_text string
#     combined_text += text

# # Print the combined text
# print(combined_text)

In [36]:
#documents = SimpleDirectoryReader('data').load_data()

from llama_index import Document

text_list = [text1]

documents = [Document(t) for t in text_list]

In [37]:
# # set number of output tokens
# num_output = 500
# # set maximum input size
# max_input_size = 512
# # set maximum chunk overlap
# max_chunk_overlap = 15


# prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
     

In [38]:
#index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index = GPTListIndex.from_documents(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index.save_to_disk('index.json')

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


In [39]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [40]:
#response = index.query( "What's the cost of Whisper model?") 


In [51]:
query_engine = index.as_query_engine()
prompt = "who was the founder McDonalds"
response = query_engine.query(prompt)
print(response)


McDonalds was founded by James McDonald
